# Setup

In [1]:
!pip install timm
import timm 
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 20.2 MB/s 


In [2]:
# to use kaggle api #
!pip install kaggle

# kaggle API creds #
creds = '{"username":"victorbahlangene","key":"2cd1abe2f1c35ea3060e6a2f18a055ea"}'

from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)


try: import fastkaggle
except ModuleNotFoundError:
    !pip install -q fastkaggle

from fastkaggle import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# competion name #
comp = 'sign-language-image-classification'

# path to where data will be downloaded #
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')

from fastai.vision.all import *
set_seed(42)

100%|██████████| 214M/214M [00:07<00:00, 30.9MB/s]


In [4]:
path

Path('sign-language-image-classification')

In [5]:
path.ls()

(#4) [Path('sign-language-image-classification/train.csv'),Path('sign-language-image-classification/submit.csv'),Path('sign-language-image-classification/images'),Path('sign-language-image-classification/test.csv')]

# Data 

In [6]:
# train dataframe #
df = pd.read_csv(path/"train.csv")
df

,id,label
0,b1627fb5d6af156b41c46704bc041416.jpg,8
1,bf61d61ec5f282372b76311e2594b94f.jpg,13
2,1e963951fb29f4f37d9a153e14533d80.jpg,17
3,4aec4a36181bd02af6040e3b4eef8378.jpg,14
4,78740bc6ad7b6ca6fe572a580ffa65e0.jpg,16
...,...,...
1995,ab98e8288691368429373c8ce97464bb.jpg,17
1996,8554d163909096427e3197bdfe95ab5c.jpg,7
1997,9c86ecf81893b65ee9ac478f1351ca8c.jpg,14
1998,36398d5613e8b7e5515b37d10f7b8bd9.jpg,19


In [7]:
# test dataframe #
tst_df = pd.read_csv(path/"test.csv")
tst_df

,id
0,5f3540c43f86dab2c9a7c87834311ee3.jpg
1,7f14db51f1994bce42972c7fe111f5cd.jpg
2,092908960a3d4dccbe06a60b71dcdd98.jpg
3,542ed4f107f9472457b47a436ec55e60.jpg
4,c125af0d467cffe584728aad4b426d53.jpg
...,...
1499,480bd34d7aaddb2f15d897f6fa938a1a.jpg
1500,610ecbaf1f77fcbd8dda0f664c084769.jpg
1501,f9e8adc8f33ef85d7ece4220537e9fec.jpg
1502,826744d31261d3fd854b7bf93f2952c5.jpg


In [8]:
# submission dataframe #
ss = pd.read_csv(path/"submit.csv")
ss

,id,predict
0,5f3540c43f86dab2c9a7c87834311ee3.jpg,NaN
1,7f14db51f1994bce42972c7fe111f5cd.jpg,NaN
2,092908960a3d4dccbe06a60b71dcdd98.jpg,NaN
3,542ed4f107f9472457b47a436ec55e60.jpg,NaN
4,c125af0d467cffe584728aad4b426d53.jpg,NaN
...,...,...
1499,480bd34d7aaddb2f15d897f6fa938a1a.jpg,NaN
1500,610ecbaf1f77fcbd8dda0f664c084769.jpg,NaN
1501,f9e8adc8f33ef85d7ece4220537e9fec.jpg,NaN
1502,826744d31261d3fd854b7bf93f2952c5.jpg,NaN


**Test Files**

In [9]:
doc(get_image_files)

get_image_files(path, recurse=True, folders=None)
Get image files in `path` recursively, only in `folders`, if specified.

To get a prettier result with hyperlinks to source code and documentation, install nbdev: pip install nbdev


In [10]:
tst_files = get_image_files(path/"images/images/test").sorted()
tst_files

(#1504) [Path('sign-language-image-classification/images/images/test/003a97d7ca88c5fe85b7b2d812705f10.jpg'),Path('sign-language-image-classification/images/images/test/0050374d197dd3cf0c112ca1d871a12f.jpg'),Path('sign-language-image-classification/images/images/test/0072b156fd15cc20fe1ad4d33c54902c.jpg'),Path('sign-language-image-classification/images/images/test/0085e3a04af015b51b0c6ddf81638ee9.jpg'),Path('sign-language-image-classification/images/images/test/008ff2af62e264c4b04799faef6e2a8e.jpg'),Path('sign-language-image-classification/images/images/test/009cfd592acb43f9bdb5d18b81b2804c.jpg'),Path('sign-language-image-classification/images/images/test/00bdffc5fa8920a8009adfd47ab6851c.jpg'),Path('sign-language-image-classification/images/images/test/00d134dadbc2f1896873aba12801d604.jpg'),Path('sign-language-image-classification/images/images/test/00dc759fa72b37dab0ea5641ff90180b.jpg'),Path('sign-language-image-classification/images/images/test/00eb1ee095a182502c371f6031af1144.jpg')..

# Memory and gradient accumulation

sample the training data so as to test the memory allocation for different models

In [11]:
df["label"].value_counts()

8     100
13    100
10    100
5     100
6     100
15    100
0     100
2     100
7     100
12    100
1     100
18    100
19    100
4     100
3     100
9     100
16    100
14    100
17    100
11    100
Name: label, dtype: int64

data frame of only 'labels' == 8

In [12]:
df[df["label"]==8]

,id,label
0,b1627fb5d6af156b41c46704bc041416.jpg,8
21,e012b8e9ee86894c946477b0f58b350d.jpg,8
38,97ea7b1cfecca676535e3512b0229189.jpg,8
41,6c3645247a85151f53067fd45eb2f565.jpg,8
55,af310a9475af85c0ad5d2cc99b9973ef.jpg,8
...,...,...
1922,0ade4e4b87870f6bd0bba303a38f2189.jpg,8
1957,75afb68d396396993b2ea367ce26112f.jpg,8
1975,4332a8ab26f93e81f88e53db09eef2e6.jpg,8
1980,7d0995a0a80d5799b35658215df1ec37.jpg,8


In [13]:
df_tst_cuda = df[df["label"]==8]

## Clear GPU cache

In [14]:
! pip install pynvml

import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.8 MB/s 


In [15]:
report_gpu()

GPU:0
no processes are running


## Train function

**structure of databloack and dataloaders**

```python
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(valid_pct=0.2, seed=42),
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms=Resize(480, method='squish'),
                   batch_tfms=aug_transforms(size=128, min_scale=0.75))

dls = dblock.dataloaders(trn_df)
```

In [16]:
def train(arch, size, df, item=Resize(480, method=ResizeMethod.Pad, pad_mode=PadMode.Zeros), accum=1, finetune=True, epochs=12):
  """function to help in the calculation of memory used by each model
    Args:
      arch: model architecture
      

  """
  def get_x(r): return path/"images/images/train"/r["id"]
  def get_y(r): return r["label"]
  dls = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  splitter=RandomSplitter(valid_pct=0.2),
                  get_x=get_x, 
                  get_y=get_y,
                  item_tfms=item,
                  batch_tfms=aug_transforms(size=size, min_scale=0.75)).dataloaders(df, bs=32//accum)

  cbs = GradientAccumulation(64) if accum else []                
  learn = vision_learner(dls, arch, metrics=accuracy, cbs=cbs).to_fp16()
  
  if finetune:
    learn.fine_tune(epochs, 0.001)
    tst_files = get_image_files(path/"images/images/test").sorted()
    tta_list,_ = learn.tta(dl=dls.test_dl(tst_files))
    return learn, tta_list
  else:
    learn.unfreeze()
    learn.fit_one_cycle(epochs, 0.01)


## Checking memory use

**`convnext_large_in22k`**

In [ ]:
train("convnext_large_in22k", 224, df_tst_cuda, accum=1, finetune=False, epochs=1)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.000000,0.000000,1.000000,00:08


GPU:0
process       2140 uses    14881.000 MB GPU memory


In [ ]:
train("convnext_large_in22k", 224, df_tst_cuda, accum=2, finetune=False, epochs=1)
report_gpu()

epoch,train_loss,valid_loss,accuracy,time
0,0.000000,0.000000,1.000000,00:04


GPU:0
process       2140 uses     9473.000 MB GPU memory


In [ ]:
train("convnext_large_in22k", (320,240), df_tst_cuda, accum=2, finetune=False, epochs=1)
report_gpu()

epoch,train_loss,valid_loss,accuracy,time
0,0.000000,0.000000,1.000000,00:03


GPU:0
process       2140 uses    12725.000 MB GPU memory


**`vit_large_patch16_224`**

In [ ]:
train("vit_large_patch16_224", 224, df_tst_cuda, accum=1, finetune=False, epochs=1)
report_gpu()

In [ ]:
train("vit_large_patch16_224", 224, df_tst_cuda, accum=2, finetune=False, epochs=1)
report_gpu()

**`swinv2_large_window12_192_22k`**

In [ ]:
train("swinv2_large_window12_192_22k", 192, df_tst_cuda, accum=2, finetune=False, epochs=1)
report_gpu()

In [ ]:
train("swinv2_large_window12_192_22k", 192, df_tst_cuda, accum=4, finetune=False, epochs=1)
report_gpu()

**`swin_large_patch4_window7_224`**

In [ ]:
train("swin_large_patch4_window7_224", 224, df_tst_cuda, accum=2, finetune=False, epochs=1)
report_gpu()

## Mount google drive

In [17]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


### Save array to gdrive

Create path to location on the gdrive

In [ ]:
# file_save_name = "tta_resnet.pkl"
# path_tta_res = f"/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/{file_save_name}"

In [ ]:
# path_tta_res

'/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_resnet.pkl'

save list with fastai method `save_pickle`

In [ ]:
# save_pickle(path_tta_res, tta_resnet)

### Open saved pickle file

In [ ]:
# import pickle
# # drive.mount('/content/drive')
# array_path = "/content/TTA_RES.pkl"

# with open(array_path, "rb") as file:
#   res_2_inf = pickle.load(file)


# Running models

In [ ]:
res = 480

In [ ]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [ ]:
models

{'convnext_large_in22k': {(Resize -- {'size': (480, 480), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}:
   encodes: (Image,object) -> encodes
   (TensorBBox,object) -> encodes
   (TensorPoint,object) -> encodes
   decodes: , (320, 224))},
 'vit_large_patch16_224': {(Resize -- {'size': (480, 480), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}:
   encodes: (Image,object) -> encodes
   (TensorBBox,object) -> encodes
   (TensorPoint,object) -> encodes
   decodes: ,
   224),
  (Resize -- {'size': (480, 480), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}:
   encodes: (Image,object) -> encodes
   (TensorBBox,object) -> encodes
   (TensorPoint,object) -> encodes
   decodes: , 224)},
 'swinv2_large_window12_192_22k': {(Resize -- {'size': (480, 480), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}:
   encodes: (Image,object) -> encodes
   (TensorBBox,object) -> encodes
   (TensorP

## `convnext_large_in22k` Model

In [ ]:
learn_convnext_large_in22k, tta_convnext_large_in22k = train("convnext_large_in22k", 224, df, accum=2)
report_gpu()

In [ ]:
tta_convnext_large_in22k.shape

torch.Size([1504, 20])

In [ ]:
tta_convnext_large_in22k.argmax(dim=1)

TensorBase([15, 18,  2,  ...,  5,  5,  0])

In [ ]:
tta_convnext_large_in22k.argmax(dim=1).unique()


TensorBase([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19])

In [ ]:
tta_convnext_large_in22k.argmax(dim=1).shape


torch.Size([1504])

### Save `convnext_large_in22k` TTA

In [ ]:
file_save_name = "tta_convnext_large_in22k.pkl"
path_tta_res = f"/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/{file_save_name}"
path_tta_res

'/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_convnext_large_in22k.pkl'

In [ ]:
save_pickle(path_tta_res, tta_convnext_large_in22k)

## `vit_large_patch16_224` Model

In [19]:
learn_vit_large_patch16_224, tta_vit_large_patch16_224 = train("vit_large_patch16_224", 224, df, accum=2)
report_gpu()


epoch,train_loss,valid_loss,accuracy,time
0,3.433413,1.852171,0.410000,01:04


epoch,train_loss,valid_loss,accuracy,time
0,2.259027,1.487343,0.547500,01:18
1,1.837614,0.896647,0.742500,01:17
2,1.163008,0.394851,0.872500,01:17
3,0.608036,0.225952,0.925000,01:17
4,0.387979,0.150458,0.952500,01:17
5,0.249734,0.103239,0.970000,01:17
6,0.171358,0.105692,0.970000,01:17
7,0.152230,0.080721,0.980000,01:17
8,0.105785,0.072069,0.977500,01:17
9,0.104234,0.073637,0.977500,01:17


GPU:0
process       3070 uses    11275.000 MB GPU memory


In [23]:
report_gpu()

GPU:0
process       3070 uses     6811.000 MB GPU memory


In [20]:
print(tta_vit_large_patch16_224.shape)

torch.Size([1504, 20])


In [24]:
print(tta_vit_large_patch16_224.argmax(dim=1))

tensor([15,  1,  2,  ...,  5,  5,  0])


In [25]:
print(tta_vit_large_patch16_224.argmax(dim=1).unique())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])


In [26]:
print(tta_vit_large_patch16_224.argmax(dim=1).shape)

torch.Size([1504])


### save `vit_large_patch16_224` Model

In [27]:
file_save_name = "tta_vit_large_patch16_224.pkl"
path_tta_res = f"/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/{file_save_name}"
path_tta_res



'/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_vit_large_patch16_224.pkl'

In [28]:
save_pickle(path_tta_res, tta_vit_large_patch16_224)

## `swinv2_large_window12_192_22k` Model

In [29]:
learn_swinv2_large_window12_192_22k, tta_swinv2_large_window12_192_22k = train("swinv2_large_window12_192_22k", 192, df, accum=2)
report_gpu()


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v2.0.0/swinv2_large_patch4_window12_192_22k.pth" to /root/.cache/torch/hub/checkpoints/swinv2_large_patch4_window12_192_22k.pth


epoch,train_loss,valid_loss,accuracy,time
0,3.126429,1.091888,0.632500,00:49


epoch,train_loss,valid_loss,accuracy,time
0,1.724349,0.840936,0.732500,00:57
1,1.381500,0.525181,0.845000,00:57
2,0.947155,0.286991,0.927500,00:57
3,0.638354,0.176408,0.962500,00:57
4,0.427618,0.102779,0.970000,00:57
5,0.310500,0.084011,0.980000,00:57
6,0.262002,0.054428,0.987500,00:57
7,0.190134,0.036531,0.992500,00:57
8,0.177720,0.030650,0.992500,00:57
9,0.172843,0.035878,0.987500,00:57


GPU:0
process       3070 uses    14261.000 MB GPU memory


In [30]:
print(tta_swinv2_large_window12_192_22k.shape)

torch.Size([1504, 20])


In [31]:
print(tta_swinv2_large_window12_192_22k.argmax(dim=1))

tensor([15,  1,  2,  ...,  5,  5,  0])


In [32]:
print(tta_swinv2_large_window12_192_22k.argmax(dim=1).unique())

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])


In [33]:
print(tta_swinv2_large_window12_192_22k.argmax(dim=1).shape)

torch.Size([1504])


### save `swinv2_large_window12_192_22k` Model

In [34]:
file_save_name = "tta_swinv2_large_window12_192_22k.pkl"
path_tta_res = f"/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/{file_save_name}"
path_tta_res

save_pickle(path_tta_res, tta_swinv2_large_window12_192_22k)

## `swin_large_patch4_window7_224` Model

In [18]:
learn_swin_large_patch4_window7_224, tta_swin_large_patch4_window7_224 = train("swin_large_patch4_window7_224", 224, df, accum=2)
report_gpu()


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth


epoch,train_loss,valid_loss,accuracy,time
0,3.172257,1.505366,0.505000,00:47


epoch,train_loss,valid_loss,accuracy,time
0,1.952710,1.241083,0.605000,00:54
1,1.707655,0.876543,0.715000,00:53
2,1.289687,0.593416,0.800000,00:53
3,0.935896,0.443196,0.850000,00:52
4,0.750860,0.345057,0.910000,00:53
5,0.564990,0.241439,0.922500,00:53
6,0.455121,0.216580,0.920000,00:53
7,0.409807,0.176307,0.952500,00:53
8,0.372800,0.150989,0.957500,00:53
9,0.321210,0.147354,0.955000,00:53


GPU:0
process       2354 uses     8455.000 MB GPU memory


In [19]:
print(tta_swin_large_patch4_window7_224.shape)
print(tta_swin_large_patch4_window7_224.argmax(dim=1))
print(tta_swin_large_patch4_window7_224.argmax(dim=1).unique())
print(tta_swin_large_patch4_window7_224.argmax(dim=1).shape)

torch.Size([1504, 20])
tensor([15,  1,  2,  ...,  5,  5,  0])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
torch.Size([1504])


### save `swin_large_patch4_window7_224` Model

In [20]:
file_save_name = "tta_swin_large_patch4_window7_224.pkl"
path_tta_res = f"/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/{file_save_name}"
path_tta_res

save_pickle(path_tta_res, tta_swin_large_patch4_window7_224)

In [21]:
report_gpu()

GPU:0
process       2354 uses     4599.000 MB GPU memory


# Ensembling

## Loading TTA arrays

In [22]:
test_path = "/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/TTA_RES.pkl"

In [23]:
import pickle
# drive.mount('/content/drive')
# array_path = "/content/TTA_RES.pkl"

with open(test_path, "rb") as file:
  res_inf = pickle.load(file)
  file.close()


In [25]:
res_inf

[(TensorBase([[4.2232e-04, 6.4454e-04, 2.3363e-01,  ..., 2.5377e-04,
               7.0769e-05, 2.5154e-01],
              [3.8991e-02, 4.6639e-01, 6.2257e-04,  ..., 2.3955e-04,
               4.4904e-01, 6.4602e-05],
              [4.3255e-02, 6.4383e-02, 3.0425e-02,  ..., 3.2883e-01,
               7.9076e-02, 2.7413e-03],
              ...,
              [2.0888e-03, 6.9797e-04, 1.1963e-03,  ..., 2.9960e-02,
               1.2236e-04, 1.3778e-02],
              [4.2559e-06, 1.7445e-06, 2.2864e-06,  ..., 1.2538e-02,
               5.1878e-07, 1.5011e-03],
              [5.8910e-01, 4.3467e-02, 2.0875e-03,  ..., 3.3854e-03,
               1.9407e-02, 2.4560e-03]]), None)]

### Loading specific models from gdrive

In [26]:
convext_large_path = "/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_convnext_large_in22k.pkl"
swin_large_patch4_path = "/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_swin_large_patch4_window7_224.pkl"
swinv2_large_window_path = "/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_swinv2_large_window12_192_22k.pkl"
vit_large_patch16_path = "/content/gdrive/MyDrive/Fastai notebooks/fastai course22/notebook with new dataset/Vision/Sign Language Image Classification/tta_vit_large_patch16_224.pkl"

convext_large model preds

In [27]:
with open(convext_large_path, "rb") as file:
  convext_large_path_inf = pickle.load(file)
  file.close()

swin_large_patch4 model preds

In [28]:
with open(swin_large_patch4_path, "rb") as file:
  swin_large_patch4_path_inf = pickle.load(file)
  file.close()

swinv2_large_window model preds

In [29]:
with open(swinv2_large_window_path, "rb") as file:
  swinv2_large_window_path_inf = pickle.load(file)
  file.close()

vit_large_patch16 model preds

In [30]:
with open(vit_large_patch16_path, "rb") as file:
  vit_large_patch16_path_inf = pickle.load(file)
  file.close()

Check Pred shapes

In [31]:
convext_large_path_inf.shape

torch.Size([1504, 20])

In [32]:
swin_large_patch4_path_inf.shape

torch.Size([1504, 20])

In [33]:
swinv2_large_window_path_inf.shape

torch.Size([1504, 20])

In [34]:
vit_large_patch16_path_inf.shape

torch.Size([1504, 20])

list of model preds

In [40]:
model_preds = [convext_large_path_inf, swin_large_patch4_path_inf, swinv2_large_window_path_inf, vit_large_patch16_path_inf]


### Ensemble Calculation

In [73]:
(convext_large_path_inf+swin_large_patch4_path_inf+swinv2_large_window_path_inf+vit_large_patch16_path_inf).shape

torch.Size([1504, 20])

In [74]:
ens_preds = (convext_large_path_inf+swin_large_patch4_path_inf+swinv2_large_window_path_inf+vit_large_patch16_path_inf)/4
ens_preds.shape

torch.Size([1504, 20])

.argmax(dim=1) gives the actual prediction index

In [80]:
ens_preds.argmax(dim=1), len(ens_preds.argmax(dim=1)), ens_preds.argmax(dim=1).unique(), (ens_preds.argmax(dim=1).unique()).shape

(TensorBase([15,  1,  2,  ...,  5,  5,  0]),
 1504,
 TensorBase([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19]),
 torch.Size([20]))

In [77]:
ens_preds.argmax(dim=1).shape

torch.Size([1504])

In [81]:
idxs = ens_preds.argmax(dim=1)

idxs.shape, len(idxs.unique())

(torch.Size([1504]), 20)

# Submit

In [85]:
vocab = np.array(learn_swin_large_patch4_window7_224.dls.vocab)
vocab

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [86]:
results = pd.Series(vocab[idxs], name="idxs")
results.head()

0    15
1     1
2     2
3     4
4     9
Name: idxs, dtype: int64

In [87]:
result_values = results.values

In [89]:
tst_files

(#1504) [Path('sign-language-image-classification/images/images/test/003a97d7ca88c5fe85b7b2d812705f10.jpg'),Path('sign-language-image-classification/images/images/test/0050374d197dd3cf0c112ca1d871a12f.jpg'),Path('sign-language-image-classification/images/images/test/0072b156fd15cc20fe1ad4d33c54902c.jpg'),Path('sign-language-image-classification/images/images/test/0085e3a04af015b51b0c6ddf81638ee9.jpg'),Path('sign-language-image-classification/images/images/test/008ff2af62e264c4b04799faef6e2a8e.jpg'),Path('sign-language-image-classification/images/images/test/009cfd592acb43f9bdb5d18b81b2804c.jpg'),Path('sign-language-image-classification/images/images/test/00bdffc5fa8920a8009adfd47ab6851c.jpg'),Path('sign-language-image-classification/images/images/test/00d134dadbc2f1896873aba12801d604.jpg'),Path('sign-language-image-classification/images/images/test/00dc759fa72b37dab0ea5641ff90180b.jpg'),Path('sign-language-image-classification/images/images/test/00eb1ee095a182502c371f6031af1144.jpg')..

In [88]:
ss

,id,predict
0,5f3540c43f86dab2c9a7c87834311ee3.jpg,NaN
1,7f14db51f1994bce42972c7fe111f5cd.jpg,NaN
2,092908960a3d4dccbe06a60b71dcdd98.jpg,NaN
3,542ed4f107f9472457b47a436ec55e60.jpg,NaN
4,c125af0d467cffe584728aad4b426d53.jpg,NaN
...,...,...
1499,480bd34d7aaddb2f15d897f6fa938a1a.jpg,NaN
1500,610ecbaf1f77fcbd8dda0f664c084769.jpg,NaN
1501,f9e8adc8f33ef85d7ece4220537e9fec.jpg,NaN
1502,826744d31261d3fd854b7bf93f2952c5.jpg,NaN


In [90]:
ss.sort_values("id", ascending=True, inplace=True)

In [91]:
ss

,id,predict
1296,003a97d7ca88c5fe85b7b2d812705f10.jpg,NaN
834,0050374d197dd3cf0c112ca1d871a12f.jpg,NaN
634,0072b156fd15cc20fe1ad4d33c54902c.jpg,NaN
1410,0085e3a04af015b51b0c6ddf81638ee9.jpg,NaN
931,008ff2af62e264c4b04799faef6e2a8e.jpg,NaN
...,...,...
927,ff70c770641d8b6062a666ad07030762.jpg,NaN
157,ff8469ea510473a3c4ac271e82d15a87.jpg,NaN
41,ff8799c16f7ec6e704ee8f29d8dfdcb7.jpg,NaN
1032,ffa44bfbce1fdecd9302524a6cd9bb37.jpg,NaN


In [92]:
ss["predict"] = result_values

In [93]:
ss

,id,predict
1296,003a97d7ca88c5fe85b7b2d812705f10.jpg,15
834,0050374d197dd3cf0c112ca1d871a12f.jpg,1
634,0072b156fd15cc20fe1ad4d33c54902c.jpg,2
1410,0085e3a04af015b51b0c6ddf81638ee9.jpg,4
931,008ff2af62e264c4b04799faef6e2a8e.jpg,9
...,...,...
927,ff70c770641d8b6062a666ad07030762.jpg,4
157,ff8469ea510473a3c4ac271e82d15a87.jpg,7
41,ff8799c16f7ec6e704ee8f29d8dfdcb7.jpg,5
1032,ffa44bfbce1fdecd9302524a6cd9bb37.jpg,5


In [94]:
ss.sort_index(inplace=True)

In [95]:
ss

,id,predict
0,5f3540c43f86dab2c9a7c87834311ee3.jpg,4
1,7f14db51f1994bce42972c7fe111f5cd.jpg,18
2,092908960a3d4dccbe06a60b71dcdd98.jpg,19
3,542ed4f107f9472457b47a436ec55e60.jpg,5
4,c125af0d467cffe584728aad4b426d53.jpg,1
...,...,...
1499,480bd34d7aaddb2f15d897f6fa938a1a.jpg,17
1500,610ecbaf1f77fcbd8dda0f664c084769.jpg,3
1501,f9e8adc8f33ef85d7ece4220537e9fec.jpg,9
1502,826744d31261d3fd854b7bf93f2952c5.jpg,10


**to_csv**

In [96]:
ss.to_csv('submission3.csv', index=False)
!head submission3.csv

id,predict
5f3540c43f86dab2c9a7c87834311ee3.jpg,4
7f14db51f1994bce42972c7fe111f5cd.jpg,18
092908960a3d4dccbe06a60b71dcdd98.jpg,19
542ed4f107f9472457b47a436ec55e60.jpg,5
c125af0d467cffe584728aad4b426d53.jpg,1
dac637220376b2f834447c515148e301.jpg,6
c94daf6d789e53fd933cb26cedf00678.jpg,15
559cbbb343a27ec2839477c53b0ba65b.jpg,2
4a6bd7247218fe3cd2001a634b1fd6df.jpg,8


In [97]:
if not iskaggle:
  from kaggle import api
  api.competition_submit_cli('submission3.csv', 'v6 ensembles victor', comp)

100%|██████████| 58.0k/58.0k [00:04<00:00, 12.9kB/s]


- Score: 0.98404
- Public Score: 0.98404
- POS: 1/41. This is a very small competetion

# Submit Notebook to kaggle

```python
push_notebook(user, id, title, file, path='.', competition=None, private=True, gpu=False, internet=False)
```

In [ ]:
if not iskaggle:
  push_notebook('victorbahlangene', 'sign-language-image-classification',
                title='Sign Language Image Classification part 3_1',
                file='Sign_Language_Image_Classification_part_3_1.ipynb',
                path=".",
                competition=comp, private=False, gpu=False)